## Introduction
Joe is a new freshman at Carnegie Mellon University. It is his first time in Pittsburgh and he is not familiar with the restaurants here. He wants to explore the different dining options for different occasions. However, as a busy CS student, he doesn’t have much time to do so. As his friends, we will be helping him to analyze and visualize Pittsburgh restaurant data using the Yelp Fusion API. We will be specifically focusing on the different characteristics (i.e. price, popularity, cuisine distribution, etc.) of restaurants in different regions of Pittsburgh. We will also create visualizations that Joe can use to find restaurants for his needs quickly and easily. 

## Installing Libraries
Let us first install the various library that we will use to analysis Pittsburgh restaurants for Joe. We can install the libraries by running the following commands.
  ``` 
  $ python -m pip install pandas 

  $ python -m pip install requests

  $ python -m pip install time
  ```

In [21]:
import pandas as pd
import requests
import time

In [2]:
API_KEY = "55OogUtwnlPBe3XofvQBDpHxADO1WaxgA1JFjegDtimMsoX6oqTkVJ6Gx2ry7g07D_qxm3Hdnk_cAziTh_so6rwjekkLoNDZrvTVm2_nJLuida5u70FgaFSYQrz5YXYx"

## Collecting Yelp Restaurant Data


In [22]:
def yelp_search(query):
    """
    Make an authenticated request to the Yelp API.

    Args:
        query (string): Search term

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the query
        businesses (list): list of dicts representing each business
    """

    payload = {'location': query}
    headers = {'Authorization': 'Bearer '+ API_KEY}
    response = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=payload)
    response.raise_for_status()
    json = response.json()

    return json['total'], json['businesses']

In [56]:
def all_restaurants(query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """

    payload = {'location': query, 'categories': 'restaurants', 'limit': 50}
    headers = {'Authorization': 'Bearer '+ API_KEY}
    offset = 0
    res = []
    
    # call endpoint until no additional data is received
    while(offset < 1000):
        payload['offset'] = offset
        try:
            response = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=payload)
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            break
        
        json = response.json()

        if len(json['businesses']) < 1:
            break

        offset += len(json['businesses'])
        res.extend(json['businesses'])
        time.sleep(0.5)
    return pd.json_normalize(res)

In [70]:
from tqdm import tqdm

master_df = pd.DataFrame()
pittsburgh_zipcodes = [15106, 15120, 15201, 15203, 15204, 15205, 15206, 15207, 15208, 15210, 15211, 15212, 15213, 15214, 15215, 15216, 15217, 15218, 15219, 15220, 15221, 15222, 15224, 15226, 15227, 15232, 15233, 15234, 15235, 15236, 15238, 15260, 15290]

# est. runtime 10 min
for zip in tqdm(pittsburgh_zipcodes):
  df = all_restaurants(zip)
  # strict checking of zip code, can comment out for more data
  df = df[df['location.zip_code'] == str(zip)]
  master_df = pd.concat([master_df, df])

master_df = master_df.drop_duplicates(subset=['id'])


100%|██████████| 33/33 [10:34<00:00, 19.23s/it]


In [71]:
master_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,dYlBcBLp3KY5Tc78d8k0ow,brick-n-mortar-kitchen-and-catering-heidelberg-2,Brick N' Mortar Kitchen & Catering,https://s3-media2.fl.yelpcdn.com/bphoto/iV5ccg...,False,https://www.yelp.com/biz/brick-n-mortar-kitche...,171,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,[delivery],...,40.391533,-80.087645,1709 E Railroad St,,None,Heidelberg,15106,US,PA,"[1709 E Railroad St, Heidelberg, PA 15106]"
1,jhGbILmSQ2TQvpJWug49Ew,carnegie-coffee-company-carnegie,Carnegie Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/xltDkT...,False,https://www.yelp.com/biz/carnegie-coffee-compa...,195,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,[delivery],...,40.407761,-80.086108,132 E Main St,,,Carnegie,15106,US,PA,"[132 E Main St, Carnegie, PA 15106]"
4,eMIX3ldCrPJymVenND0sDg,one-thirty-one-east-carnegie,One Thirty One East,https://s3-media3.fl.yelpcdn.com/bphoto/GRXgzK...,False,https://www.yelp.com/biz/one-thirty-one-east-c...,191,"[{'alias': 'newamerican', 'title': 'American (...",4.5,[delivery],...,40.408010,-80.086150,131 E Main St,,,Carnegie,15106,US,PA,"[131 E Main St, Carnegie, PA 15106]"
5,dP8_X1zpSo3rGarHMyzQ1A,leogreta-carnegie,LeoGreta,https://s3-media1.fl.yelpcdn.com/bphoto/IGU5Vt...,False,https://www.yelp.com/biz/leogreta-carnegie?adj...,166,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,[delivery],...,40.404324,-80.091270,301 W Main St,,None,Carnegie,15106,US,PA,"[301 W Main St, Carnegie, PA 15106]"
7,alfg8YykqggAUbeyd2nzmw,insurrection-aleworks-heidelberg,Insurrection AleWorks,https://s3-media2.fl.yelpcdn.com/bphoto/rYSQoY...,False,https://www.yelp.com/biz/insurrection-aleworks...,145,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.5,[delivery],...,40.392452,-80.087402,1635 E Railroad St,,,Heidelberg,15106,US,PA,"[1635 E Railroad St, Heidelberg, PA 15106]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,2XZ4g_ps_H55CmxFs4Pu8A,chilis-pittsburgh,Chili's,https://s3-media2.fl.yelpcdn.com/bphoto/wpQNgf...,False,https://www.yelp.com/biz/chilis-pittsburgh?adj...,52,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",2.5,[],...,40.486924,-79.891602,855 Freeport Rd,None,None,Pittsburgh,15238,US,PA,"[855 Freeport Rd, Pittsburgh, PA 15238]"
864,I-D4qEtJJ4okSLrLKxMMRw,burger-king-pittsburgh-13,Burger King,https://s3-media4.fl.yelpcdn.com/bphoto/eXdff-...,False,https://www.yelp.com/biz/burger-king-pittsburg...,9,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,[delivery],...,40.538527,-79.833244,2799 Freeport Rd,None,None,Pittsburgh,15238,US,PA,"[2799 Freeport Rd, Pittsburgh, PA 15238]"
866,i851zomQeoY7gbrkFcvM7Q,dennys-pittsburgh-4,Denny's,https://s3-media2.fl.yelpcdn.com/bphoto/-JtXII...,False,https://www.yelp.com/biz/dennys-pittsburgh-4?a...,30,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",2.5,"[pickup, delivery]",...,40.538324,-79.834410,2701 Freeport Rd,,,Pittsburgh,15238,US,PA,"[2701 Freeport Rd, Pittsburgh, PA 15238]"
975,2A9gJkmWbBVoh6kK0jDlxA,la-pizza-bella-pittsburgh,La Pizza Bella,,False,https://www.yelp.com/biz/la-pizza-bella-pittsb...,6,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"[pickup, delivery]",...,40.492817,-79.862587,268 Freeport Rd,Ste 1,None,Pittsburgh,15238,US,PA,"[268 Freeport Rd, Ste 1, Pittsburgh, PA 15238]"
